In [0]:
import torch
from torch import nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [0]:
data = pd.read_csv('data_final3.csv')
data.head()

,PostUploadTime,Headline,Flair,Rating,UserName,UserPostKarma,UserCommentKarma,Id,PostSource,NumComments,Comments,PostBody
0,1.587106e+09,Man Arrested For Allegedly Spitting On Manipur...,Non-Political,111,ReallyRedditLover,1008.0,116.0,g2ph3l,https://www.ndtv.com/mumbai-news/man-arrested-...,7,">The accused was identified as **Amir Khan** ,...",NaN
1,1.587106e+09,Bought air tickets during lockdown? Full refun...,Coronavirus,8,ReallyRedditLover,1008.0,116.0,g2pg1d,https://timesofindia.indiatimes.com/business/i...,2,"So, If I booked the ticket on 21st for 26th Ma...",NaN
2,1.587105e+09,Bollywood Nazis: A Tale of Two Sisters,Politics,19,iSalaamU,32174.0,14938.0,g2pd5o,https://www.reddit.com/r/india/comments/g2pd5o...,7,Woah so permanent ban right ? Not temporary .....,"Yesterday, Twitter permanently suspended the a..."
3,1.587105e+09,"On this occasion of Rongāli bihu, I'd love to ...",Non-Political,10,deboo117,8370.0,2700.0,g2p97t,https://open.spotify.com/playlist/21uBCM64wmKV...,0,NaN,NaN
4,1.587104e+09,A Tweet Asking For Muslims And Journalists To ...,Coronavirus,1,NaN,NaN,NaN,g2ovv5,https://www.buzzfeednews.com/article/pranavdix...,1,NaN,[deleted]


# Flair Classification

1. Using all the numerical columns as features(independent variable). Predicting Flair class using only numerical data<br>
  model: ***XGBoost***

In [0]:
#Data Pre Processing
df = data.filter(['Flair', 'Rating', 'UserPostKarma', 'UserCommentKarma', 'NumComments'], axis = 1)
df.head()

,Flair,Rating,UserPostKarma,UserCommentKarma,NumComments
0,Non-Political,111,1008.0,116.0,7
1,Coronavirus,8,1008.0,116.0,2
2,Politics,19,32174.0,14938.0,7
3,Non-Political,10,8370.0,2700.0,0
4,Coronavirus,1,NaN,NaN,1


In [0]:
# assigning codes to Flair column
category = df.Flair.astype('category')
print(category.unique())

# assigns category codes to data based on lexical order
df.Flair = category.cat.codes

[Non-Political, Coronavirus, Politics, Policy/Economy, Food, Science/Technology, Business/Finance, Photography, Sports]
Categories (9, object): [Non-Political, Coronavirus, Politics, Policy/Economy, ..., Science/Technology,
                         Business/Finance, Photography, Sports]


In [0]:
df.head()

,Flair,Rating,UserPostKarma,UserCommentKarma,NumComments
0,3,111,1008.0,116.0,7
1,1,8,1008.0,116.0,2
2,6,19,32174.0,14938.0,7
3,3,10,8370.0,2700.0,0
4,1,1,NaN,NaN,1


In [0]:
# 1. Removing nan rows
df1 = df.dropna()

# 2. replacing Nan values with the mean of its row 
df2 = df.drop('Flair', axis=1).apply(lambda row: row.fillna(row.mean()), axis=1)
df2['Flair'] = df.Flair

,Rating,UserPostKarma,UserCommentKarma,NumComments,Flair
0,111.0,1008.0,116.0,7.0,3
1,8.0,1008.0,116.0,2.0,1
2,19.0,32174.0,14938.0,7.0,6
3,10.0,8370.0,2700.0,0.0,3
4,1.0,1.0,1.0,1.0,1
5,25.0,17.0,0.0,21.0,6
6,3530.0,7834.0,1936.0,89.0,1
7,7.0,57.0,1013.0,4.0,1
8,0.0,1036.0,-4.0,2.0,5
9,6.0,5.5,5.5,5.0,1


In [0]:
import xgboost
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn import metrics

In [0]:
# training on dataset(dropped NaN rows)
target = 'Flair'
train_x, test_x, train_y, test_y = train_test_split(df1.drop(target, axis=1), df1[target], test_size = 0.2)

classifier = xgboost.XGBClassifier()
classifier.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
from math import sqrt
def rmse(pred, label):
  return sqrt(mse(label, pred))

We can see the accuracy is quite low. So using only numerical

In [0]:
preds = classifier.predict(test_x)
print(metrics.classification_report(test_y, preds, labels = np.arange(9)))

              precision    recall  f1-score   support

           0       0.28      0.23      0.26       209
           1       0.35      0.34      0.34       472
           2       1.00      0.04      0.07        56
           3       0.27      0.46      0.34       372
           4       0.50      0.06      0.11       113
           5       0.41      0.41      0.41       409
           6       0.39      0.49      0.43       464
           7       0.26      0.04      0.07       156
           8       0.12      0.01      0.03        67

    accuracy                           0.34      2318
   macro avg       0.40      0.23      0.23      2318
weighted avg       0.36      0.34      0.32      2318



Here on Using 

In [0]:
# training on dataset(replacin NaN cells with mean of its row)
target = 'Flair'
train_x, test_x, train_y, test_y = train_test_split(df2.drop(target, axis=1), df2[target], test_size = 0.2)

classifier.fit(train_x, train_y)

preds = classifier.predict(test_x)
print(metrics.classification_report(test_y, preds, labels = np.arange(9)))

              precision    recall  f1-score   support

           0       0.22      0.50      0.30       434
           1       0.33      0.25      0.28       614
           2       0.67      0.02      0.04       109
           3       0.23      0.33      0.27       611
           4       1.00      0.01      0.02       200
           5       0.32      0.34      0.33       487
           6       0.33      0.36      0.35       578
           7       0.31      0.01      0.03       374
           8       0.33      0.01      0.02       124

    accuracy                           0.27      3531
   macro avg       0.42      0.20      0.18      3531
weighted avg       0.34      0.27      0.24      3531



While using the dropped Nan rows data, the accuracy has comne out more than the dataset in which NaN values are replaced by their row's mean.<br>
Anyway my idea of using only numerical data as features for classification has not proved good. As the accuracy is only 35% which is not a good accuracy for a my classification task.<br>
So now i will also include the text for training and see how the result comes out